In [1]:
df1 = spark.read.csv('hdfs://default/user/erin/all_game_scores.csv', sep=',',encoding='UTF-8',comment=None, header=True,inferSchema=True)
df= spark.read.csv('hdfs://default/user/erin/team_game_data.csv', sep=',',encoding='UTF-8',comment=None, header=True,inferSchema=True)

df1.show(5)
df.show(5)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,Link,,erin,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------+---------------+------------+---------------+----------------+----------------+-------------+-----------------+----------+-------+-------+--------------+
| id|home_team_id|home_team_score|away_team_id|away_team_score|away_team_scores|home_team_scores|number_of_ots|             date|attendance|playoff| season|         notes|
+---+------------+---------------+------------+---------------+----------------+----------------+-------------+-----------------+----------+-------+-------+--------------+
|  1|           1|             93|           8|             85|              []|              []|         null|Sat, Oct 29, 1949|      null|  false|1949-50|          null|
|  2|          32|             87|          20|             89|              []|              []|         null| Tue, Nov 1, 1949|      null|  false|1949-50|          null|
|  3|          33|             71|           8|             64|              []|              []|         null| Tue, Nov 1, 1949|      null|

step1

**1.读入并组合两个csv表格**

**组合两个csv，将队伍的场次得分与队伍当场细节得分进行整合。

In [2]:
df1=df1.withColumnRenamed("id","did")
df=df.join(df1, df.game_id == df1.did)
df=df.drop("did","away_team_scores","home_team_scores")
df.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------+-------+--------------+---+----+------+----+----+-------+---+----+------+----+----+----+----+----+----+----+----+---+------------+---------------+------------+---------------+-------------+-----------------+----------+-------+-------+-----+
| id|game_id|team_id|minutes_played| fg| fga|fg_pct| fg3|fg3a|fg3_pct| ft| fta|ft_pct| orb| drb| trb| ast| stl| blk| tov|  pf|pts|home_team_id|home_team_score|away_team_id|away_team_score|number_of_ots|             date|attendance|playoff| season|notes|
+---+-------+-------+--------------+---+----+------+----+----+-------+---+----+------+----+----+----+----+----+----+----+----+---+------------+---------------+------------+---------------+-------------+-----------------+----------+-------+-------+-----+
|  1|      1|      8|           240| 29|null|  null|null|null|   null| 27|null|  null|null|null|null|null|null|null|null|  35| 85|           1|             93|           8|             85|         null|Sat, Oct 29, 1949|      null|  false

step2

**2.精炼出特殊特征**
- 1.每队的每场比赛是否胜利，如果胜利特征win为1，反之为0
- 2.将string形式的日期调整格式并转化为datetime形式
- 3.从datetime格式中提取月份，年份

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from datetime import datetime
from pyspark.sql.types import DateType
from pyspark.sql.functions import  year,month
from pyspark.sql.functions import when

def datetrans1(a,b,c):
    if a<b and c==1:
        return 0
    elif a>b and c==0:
        return 0
    else :
        return 1

def datetrans2(m):
    date_object=datetime.strptime(m, '%b%d,%Y')
    return date_object

df = df.withColumn("home", when(df.team_id == df.home_team_id,1)
                                 .otherwise(0))

func=udf(datetrans1,IntegerType())
df=df.withColumn("win",func(df.home_team_score,df.away_team_score,df.home))

spaceDeleteUDF = udf(lambda s: s.replace(" ", ""), StringType())
df=df.withColumn("date", spaceDeleteUDF(df.date))
df=df.withColumn('date', df.date[5:20])


func=udf(datetrans2,DateType())
df=df.withColumn("realdate",func(df.date))

df=df.withColumn("month",month(df.realdate))
df=df.withColumn("year",year(df.realdate))

df=df.withColumn("curwin",df["win"])
df=df.drop("minutes_played","home_team_id","home_team_score","away_team_id","away_team_score","pts","date","notes")
print('success')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

success

step 3

**3.清洗重复数据**
- 整行去重：data.distinct()这个方程可以对于完全相同的行进行去重。
- dataframe.count()方程会打印出这个列表的行数，以便于您了解去重之后的数据集行数。

In [4]:
df=df.distinct()
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

128233

In [5]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- fg: integer (nullable = true)
 |-- fga: integer (nullable = true)
 |-- fg_pct: double (nullable = true)
 |-- fg3: integer (nullable = true)
 |-- fg3a: integer (nullable = true)
 |-- fg3_pct: double (nullable = true)
 |-- ft: integer (nullable = true)
 |-- fta: integer (nullable = true)
 |-- ft_pct: double (nullable = true)
 |-- orb: integer (nullable = true)
 |-- drb: integer (nullable = true)
 |-- trb: integer (nullable = true)
 |-- ast: integer (nullable = true)
 |-- stl: integer (nullable = true)
 |-- blk: integer (nullable = true)
 |-- tov: integer (nullable = true)
 |-- pf: integer (nullable = true)
 |-- number_of_ots: integer (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- playoff: boolean (nullable = true)
 |-- season: string (nullable = true)
 |-- home: integer (nullable = false)
 |-- win: integer (nullable = true)
 |-- realdate: date (n

step4

**4.将数据类别进行转化**
- 以下这个单元格将一些列的数据种类进行了改变。您可以通过在withColumn这个方程来进行改变，在括号中填入您想要改变的列名，以及想改变成的种类形式。形式：（“列名”,dataframe["列名"].cast('type')）

In [6]:
from pyspark.sql.functions import col
colNames = df.columns
remove=['realdate','playoff','newplayoff','season']
colNames= filter(lambda i: i not in remove, colNames)
for colName in colNames:
    df = df.withColumn(colName, col(colName).cast('double'))
df=df.fillna(0)
df=df.orderBy('id')
df=df.filter(df.year>=2010)
df.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+-------+----+-----+------+----+----+-------+----+----+------+----+----+----+----+---+---+----+----+-------------+----------+-------+-------+----+---+----------+-----+------+------+
|      id|game_id|team_id|  fg|  fga|fg_pct| fg3|fg3a|fg3_pct|  ft| fta|ft_pct| orb| drb| trb| ast|stl|blk| tov|  pf|number_of_ots|attendance|playoff| season|home|win|  realdate|month|  year|curwin|
+--------+-------+-------+----+-----+------+----+----+-------+----+----+------+----+----+----+----+---+---+----+----+-------------+----------+-------+-------+----+---+----------+-----+------+------+
|101752.0|50896.0|   20.0|45.0| 88.0| 0.511|11.0|24.0|  0.458|11.0|17.0| 0.647|10.0|36.0|46.0|25.0|5.0|2.0|13.0|16.0|          1.0|   17366.0|  false|2009-10| 0.0|1.0|2010-01-01|  1.0|2010.0|   1.0|
|101753.0|50896.0|    1.0|46.0|100.0|  0.46| 7.0|27.0|  0.259| 9.0|14.0| 0.643|17.0|31.0|48.0|26.0|7.0|7.0|10.0|18.0|          1.0|   17366.0|  false|2009-10| 1.0|0.0|2010-01-01|  1.0|2010.0|   0.0|
+----

step5

**5.对每列数据进行scale**

In [7]:
from pyspark.sql.functions import stddev, mean
avgcol=['fg','fga','fg_pct','fg3','fg3a','fg3_pct','ft','fta','ft_pct','orb','drb','trb','ast','stl','blk','tov','pf','number_of_ots','attendance','win']
for i in avgcol:
    mean_amount, sttdev_amount = df.select(mean(i), stddev(i)).first()
    str1='scaled'+i
    df=df.withColumn(str1, (col(i) - mean_amount) / sttdev_amount)
    df=df.drop(i)
    df=df.withColumnRenamed(str1,i)
print('success')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

success

step6

**6.将每队每场比赛的表现数据替换成最近八场比赛的平均数据**
- 为了对比赛进行精准预测，通过function avg_previous_num_games对该队的该场比赛表现进行预测

In [8]:
teams = df.select("team_id").distinct().rdd.flatMap(lambda x: x).collect()
pdf=df.toPandas()

todaycol=['fg','fga','fg_pct','fg3','fg3a','fg3_pct','ft','fta','ft_pct','orb','drb','trb','ast','stl','blk','tov'
      ,'pf','number_of_ots','attendance','win']
def avg_previous_num_games(df,num_games):
    for col in todaycol:
        for team in teams:
            df[col].loc[df['team_id'] == team]=df[col].loc[df['team_id']==team].shift(1).rolling(num_games,min_periods=3).mean()
    return df.dropna()

newpdf=avg_previous_num_games(pdf,8)
df = spark.createDataFrame(newpdf)
print('success')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

success
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)

step7

加入elo系数

**7.将同属于同一场比赛的两队信息进行组合**
- 将dataframe分成主场球队比赛数据dataframe和客场球队比赛数据dataframe，将两个dataframe基于打同一场比赛的信息合并，得到的最终dataframe每行数据为一场比赛的主场球队数据与客场球队数据，便于横向比较两个球队的实力区别，更准确地预测比赛输赢。

In [9]:
dfhome=df.filter(df['home']==1)
dfaway=df.filter(df['home']==0)

cols2=['game_id','team_id','fg','fga','fg_pct','fg3','fg3a','fg3_pct','ft','fta','ft_pct','orb','drb','trb','ast','stl','blk','tov'
      ,'pf','number_of_ots','attendance','win']
dfaway=dfaway.select(*cols2)
for i in cols2:
    str1="new"+i
    dfaway=dfaway.withColumnRenamed(i,str1)

dfhome=dfhome.join(dfaway, dfhome.game_id == dfaway.newgame_id)
dfhome = dfhome.withColumn("game_id",dfhome['game_id'].cast('int'))
dfhome=dfhome.orderBy(dfhome['game_id'])
print('success')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

success

**8.计算每队实时elo系数**
- 根据每场比赛的输赢情况，以1600为起始elo参数，随着比赛输赢情况，让elo实时更新。
- 因使用特征参数来预测比赛结果，若将每行比赛的输赢情况及时反馈到elo特征系数上，则会使比赛结果影响特征向量，而在预测结果时，假定的情况是比赛结果未知，并特征向量不被当场比赛结果影响，只被之前的比赛结果影响。
- 因此每场比赛的team_rank先存于dict elo中，在打下场比赛时，dict中的实时更新的elo值赋予到dataframe的该行中，再根据比赛情况计算出新的team_rank,并在dict中更新参与比赛队伍的team_rank。


In [ ]:
from pyspark.sql.functions import lit
base_elo = 1600
dfhome=dfhome.withColumn("elo",lit(base_elo))
dfhome=dfhome.withColumn("newelo",lit(base_elo))
print('success')

In [24]:
import math
from pyspark.sql.types import FloatType

dfelo=dfhome.select("game_id","team_id","newteam_id","curwin","elo","newelo","season")
pdfelo=dfelo.toPandas()
oldseason = "2009-10"
elo={}
teams = df.select("team_id").distinct().rdd.flatMap(lambda x: x).collect()    
seasons = dfhome.select("season").distinct().rdd.flatMap(lambda x: x).collect()
for i in seasons:
    elo[i]={}
    for j in teams:
        elo[i][j]=1600.0

def elofunc1(pdfelo):
    for index,row in pdfelo.iterrows():
        curwin=row['curwin']
        teamid=row['team_id']
        newteamid=row['newteam_id']
        season=row['season']
        if(curwin==1.0):
            win_team=teamid
            lose_team=newteamid
        else:
            win_team=newteamid
            lose_team=teamid
        
        winner_rank = elo[season][win_team]
        loser_rank = elo[season][lose_team]
        rank_diff = winner_rank - loser_rank
        exp = (rank_diff  * -1) / 400
        odds = 1 / (1 + math.pow(10, exp))
        # Alter K based on rank
        if winner_rank < 2100:
            k =32
        elif winner_rank >= 2100 and winner_rank < 2400:
            k =24
        else:
            k =16
        new_winner_rank = round(winner_rank + (k * (1 - odds)))
        new_loser_rank = round(loser_rank + (k * (0 - odds)))
        pdfelo.loc[index,'elo']= float(elo[season][teamid])
        pdfelo.loc[index,'newelo']= float(elo[season][newteamid])
        elo[season][win_team]=new_winner_rank
        elo[season][lose_team]=new_loser_rank



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
elofunc1(pdfelo)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
dfelo = spark.createDataFrame(pdfelo)
dfelo.printSchema()
colNames = dfelo.columns
for colName in colNames:
    str1="e"+colName
    dfelo = dfelo.withColumnRenamed(colName,str1)
dfhome=dfhome.join(dfelo, dfhome.game_id==dfelo.egame_id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- game_id: long (nullable = true)
 |-- team_id: double (nullable = true)
 |-- newteam_id: double (nullable = true)
 |-- curwin: double (nullable = true)
 |-- elo: double (nullable = true)
 |-- newelo: double (nullable = true)
 |-- season: string (nullable = true)

In [35]:

dfhome=dfhome.drop("egame_id","eteam_id","enewteam_id","enewwin","eseason","newgame_id","elo","newelo","ecurwin")
dfhome.write.format("hive").mode("overwrite").saveAsTable("user_erin.nbabetting1")
print('success')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
dfhome.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: double (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- team_id: double (nullable = true)
 |-- playoff: boolean (nullable = true)
 |-- season: string (nullable = true)
 |-- home: double (nullable = true)
 |-- realdate: date (nullable = true)
 |-- month: double (nullable = true)
 |-- year: double (nullable = true)
 |-- curwin: double (nullable = true)
 |-- fg: double (nullable = true)
 |-- fga: double (nullable = true)
 |-- fg_pct: double (nullable = true)
 |-- fg3: double (nullable = true)
 |-- fg3a: double (nullable = true)
 |-- fg3_pct: double (nullable = true)
 |-- ft: double (nullable = true)
 |-- fta: double (nullable = true)
 |-- ft_pct: double (nullable = true)
 |-- orb: double (nullable = true)
 |-- drb: double (nullable = true)
 |-- trb: double (nullable = true)
 |-- ast: double (nullable = true)
 |-- stl: double (nullable = true)
 |-- blk: double (nullable = true)
 |-- tov: double (nullable = true)
 |-- pf: double (nullable = true)
 |-- number_of_o

In [44]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…